# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.mixture import GaussianMixture
from smart_open import open
from stop_words import get_stop_words
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [ ]:
newsgroups_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [ ]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [ ]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [ ]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)

In [ ]:
stop_words = get_stop_words("english")
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve", "'d", "'ll", ""]
stop_words = stop_words + stop_words_add + stop_char

In [ ]:
texts = newsgroups_df.words_nonstop.progress_apply(lambda x: x.split(' ')).tolist()
#     一文字以下の単語とstop_word, stop_charを削除
texts = [[word for word in text if len(word) > 1 if word not in stop_words] for text in texts]
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)
#     登場頻度80%以上の単語を削除
dictionary.filter_extremes(no_above=0.8)

In [ ]:
label = newsgroups_df["class"].to_numpy()

In [10]:
model_nums = 2
vector_model_num = 0

In [11]:
pred_path_root = "../../../Clustering/data/20News/doc2vec/GMM/pred/"

top_words = []
for model_num in range(model_nums):
    coherence = []
    for vector_size in tqdm(vector_sizes):
        pred_path = f"{pred_path_root}{vector_size}/normalized/{vector_model_num}.csv"
        
        

100%|██████████| 14/14 [5:09:55<00:00, 1328.28s/it]  


In [12]:
pd.DataFrame(coherences)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.260099,0.567958,0.469335,0.481532,0.532413,0.412016,0.402025,0.559057,0.434881,0.382951,0.675027,0.452253,0.272288,0.447694
1,0.541707,0.439913,0.419599,0.526837,0.443053,0.440034,0.493740,0.520011,0.405326,0.606243,0.573800,0.480668,0.630010,0.525632
